<a href="https://colab.research.google.com/github/ianzin30/IF1014-Mining/blob/main/Nomao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# CRISP_DM PHASE 1
Entendimento

https://docs.google.com/document/d/1__5aHtUM-75B60eRaHWH0kN8GaWUBLRkCMg_DDWnB0E/edit?usp=sharing



# CRISP_DM PHASE 2
Compreensão

## Inicializando dados


In [ ]:
# Executando os imports necessários para rodar o projeto

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Adquirindo features do Nomao
file_path_features = '/content/drive/MyDrive/nomao_dataset/Nomao.features'

with open(file_path_features, 'r') as file:
    content_features = file.read()

# Adquirindo data do Nomao
file_path_data = '/content/drive/MyDrive/nomao_dataset/Nomao.data'
with open(file_path_data, 'r') as file:
    content_data = file.read()

# Colocando features e data para criar o dataFrame
features = content_features.split('\n')
data = list(map(lambda x: x.split(','), content_data.split('\n')))

In [ ]:
# Inicializando DataFrame do Nomao
nomao_df = pd.DataFrame(data, columns=features)
nomao_df.head()

,id,clean_name_intersect_min,clean_name_intersect_max,clean_name_levenshtein_sim,clean_name_trigram_sim,clean_name_levenshtein_term,clean_name_trigram_term,clean_name_including,clean_name_equality,city_intersect_min,...,coordinates_long_levenshtein,coordinates_long_trigram,coordinates_long_equality,coordinates_lat_diff,coordinates_lat_levenshtein,coordinates_lat_trigram,coordinates_lat_equality,geocode_coordinates_diff,coordinates_diff,label
0,0#1,1,1,1,1,1,1,s,s,?,...,0.75,0.5,n,0.999953002209,0.777777777778,0.461538463831,n,1.0,0.999999996702,+1
1,0#2,1,0.75,0.857143,0.857143,0.894737,0.947368,s,n,?,...,0.5,0.307692319155,n,0.999982921892,0.538461538462,0.4375,n,1.0,0.999999962459,+1
2,0#3,1,1,1,1,1,1,s,s,?,...,0.666666666667,0.461538463831,n,0.999992600055,0.8,0.666666686535,n,1.0,0.999999999653,+1
3,4#5,1,0.75,0.857143,0.857143,0.842105,0.833333,s,n,1,...,0.5,0.285714298487,n,0.998604948886,0.555555555556,0.384615391493,n,0.999999947284,0.999994124065,+1
4,6#7,0,0,0.25,0,0.25,0,n,n,?,...,?,?,m,?,?,?,m,1.0,?,+1


## Entendendo melhor os dados do Nomao



Segundo o arquivo de informação do desafio da Nomao, é informado que:

First 29,104 instances have been labeled with "human prior".

Next 917 instances have been labeled using the active learning method called "marg".

Next 964 instances refer to the active method called "wmarg".

Next 995 instances refer to the active method called "wmarg5".

Next 1,985 instances refer to the active method called "rand" (random selection).

Next 163 instances refer to the active method called "baseline".

Next 167 instances refer to the active method called "nomao".

And last 170 instances refer to the active method called "tsun".


In [ ]:
nomao_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34466 entries, 0 to 34465
Columns: 120 entries, id to label
dtypes: object(120)
memory usage: 31.6+ MB


Podemos ver que o método "info", nos diz que existem 34466 linhas e 120 features, um arquivo bem grande e que condiz com o que foi informado pelo desafio.

Podemos então começar com o pré processamento de dados, verificando duplicatas, linhas com informações faltantes, inconcistência.

Vale ressaltar que no arquivo .names, é informado como é catégorizado dados incompletos, "?"


In [ ]:
nomao_df.duplicated().sum()

0

In [ ]:
nomao_df.dtypes

,0
id,object
clean_name_intersect_min,object
clean_name_intersect_max,object
clean_name_levenshtein_sim,object
clean_name_trigram_sim,object
...,...
coordinates_lat_trigram,object
coordinates_lat_equality,object
geocode_coordinates_diff,object
coordinates_diff,object


In [ ]:
# Não é necessário rodar, mas é interessante para ver a quantidade de NaN em cada coluna do Dataset
pd.set_option('display.max_rows', None)

In [ ]:
nomao_df.replace("?", pd.NA, inplace=True) # Irei transformar todos os valores "?" com o padrão de Missing value de Pandas.
x = nomao_df.isna().sum()
x.sort_values(ascending=False)
# Tem muita coluna com MUITOS valores em NaN. Por mim tira


,0
fax_diff,33570
fax_levenshtein,33570
fax_trigram,33570
website_intersect_max,32289
website_intersect_min,32289
website_trigram_sim,32289
website_levenshtein_term,32289
website_trigram_term,32289
website_levenshtein_sim,32289
countryname_levenshtein_sim,28798


In [ ]:
nomao_df['fax_diff'].head()

,fax_diff
0,?
1,?
2,?
3,?
4,?


In [ ]:
nomao_df['clean_name_intersect_']

Index(['id', 'clean_name_intersect_min', 'clean_name_intersect_max',
       'clean_name_levenshtein_sim', 'clean_name_trigram_sim',
       'clean_name_levenshtein_term', 'clean_name_trigram_term',
       'clean_name_including', 'clean_name_equality', 'city_intersect_min',
       ...
       'coordinates_long_levenshtein', 'coordinates_long_trigram',
       'coordinates_long_equality', 'coordinates_lat_diff',
       'coordinates_lat_levenshtein', 'coordinates_lat_trigram',
       'coordinates_lat_equality', 'geocode_coordinates_diff',
       'coordinates_diff', 'label'],
      dtype='object', length=120)